# task no. 4

This is the notebook taking care of the task no.4 : feature prediction.

In our analysis we have decided very different models to see how they differ from eachother in term of performance and also their explainability to see if they make sense.

For this notebook we decided to explore the nature of NeuralNetworks on the dataset.

First a preparation is due to be used.

## dataset preparation



In [1]:
import pandas as pd
import os


RACES_PATH=os.path.join("..","dataset","engineered_races.csv")

races_df=pd.read_csv(RACES_PATH)

First we binarize the columns

In [2]:
from sklearn.model_selection import train_test_split

races_df['position']=(races_df['position']>20).astype(int)
split_idx= pd.to_datetime(races_df['date']).dt.year <= 2022
races_df['date']=pd.to_datetime(races_df['date']).astype('int64')

one_hot=pd.get_dummies(races_df.select_dtypes(include=['bool','object']))
numeric=races_df.select_dtypes(include=['number'])
std_numeric= (numeric-numeric.mean())/(numeric.max()-numeric.min()).drop(columns='position')


data_set=pd.concat([one_hot,std_numeric],axis=1)

X_test_set=data_set.loc[~split_idx]

X_dev_set=data_set[split_idx]
Y_dev_set=data_set.loc[split_idx,'position']

X_train_set,X_val_set,Y_train_set,Y_val_set=train_test_split(
    X_dev_set,
    Y_dev_set,
    test_size=0.2,
    stratify=Y_dev_set,
    random_state=42
)

first a stratification can only help the generization capabilities.

Now we have to setup the task, for this kind of setting the binary cross entropy is the most appropriate given we just want to classify stuff and we are not doing any regression whatsoever.

A first test using a simple NN might be usefull in this case to see the most basic algorithm.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, initializers

initializer=initializers.HeNormal()

input_size=X_train_set.shape[1]

output_size=1

# Define the model
model = models.Sequential([
    layers.Dense(64, activation='relu', 
                 kernel_initializer=initializer,  # He initialization
                 bias_initializer=initializer,       # Bias initialized to zeros
                 input_shape=(input_size,)),                          # Input shape
    layers.Dense(32, activation='relu',
                 kernel_initializer=initializer),  # Xavier initialization
    layers.Dense(output_size, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy','f1-score']
)

n_epochs=1024

model.fit(
    X_train_set,Y_train_set,
    epochs=100
    )

2024-12-03 21:43:16.534684: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 21:43:16.721668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 21:43:16.776425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 21:43:16.782152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 21:43:16.876470: I tensorflow/core/platform/cpu_feature_guar

ModuleNotFoundError: No module named 'tensorflow.keras'